Imports

In [1]:
import f90nml
import numpy as np
from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import os
import reference_solution as refsol
from scipy.fft import rfft
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import icepinn as ip

torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

device = ip.get_device()

cuda
cuda


In [2]:
# Read in GI parameters
inputfile = "GI parameters - Reference limit cycle (for testing).nml"
GI=f90nml.read(inputfile)['GI']
nx_crystal = GI['nx_crystal']
L = GI['L']
NBAR = GI['Nbar']
NSTAR = GI['Nstar']

# Define t range (needs to be same as training file)
RUNTIME = 5
NUM_T_STEPS = RUNTIME*5 + 1

# Define initial conditions
Ntot_init = np.ones(nx_crystal)
Nqll_init = ip.get_Nqll(Ntot_init)

# Define x, t pairs for training
X_QLC = np.linspace(-L,L,nx_crystal)
t_points = np.linspace(0, RUNTIME, NUM_T_STEPS)
x, t = np.meshgrid(X_QLC, t_points)
training_set = torch.tensor(np.column_stack((x.flatten(), t.flatten()))).to(device)

In [3]:
# Define model attributes; instantiate model
model_dimensions = torch.tensor([8, 80]).to(device)
is_sf_PINN = torch.tensor(True)
model = ip.IcePINN(
	num_hidden_layers=model_dimensions[0], 
	hidden_layer_size=model_dimensions[0],
	is_sf_PINN=is_sf_PINN.item()).to(device)

# Attach model attributes as buffers so they can be saved and loaded
model.register_buffer('dimensions', model_dimensions)
model.register_buffer('is_sf_PINN', is_sf_PINN)

# Initialize model weights with HE initialization
def init_HE(m):
		if type(m) == nn.Linear:
			nn.init.kaiming_normal_(m.weight)
model.apply(init_HE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [4]:
print(training_set.shape)
print(ip.calc_cp_loss(model, training_set, ip.get_misc_params(), 10, 1, 1, False).shape)
print(ip.enforced_model(training_set, model).shape)

torch.Size([8320, 2])
torch.Size([8320, 2])
torch.Size([8320, 2])


Train model

In [5]:
MODEL_NAME = "TestPINN_hard_enforced_SF"
PATH = './models/'+MODEL_NAME

ip.train_IcePINN(
    model, 
    optimizer, 
    training_set, 
    epochs=100_000, 
    save_path=PATH, 
    print_every=2_500, 
    print_gradients=False)

Commencing PINN training for 100000 epochs.
Loss at epoch [2500/100000]: Ntot = 3664621.603, Nqll = 709125.894.
Loss at epoch [5000/100000]: Ntot = 3143277.069, Nqll = 161284.453.
Loss at epoch [7500/100000]: Ntot = 3193804.412, Nqll = 107407.177.
Loss at epoch [10000/100000]: Ntot = 3271224.107, Nqll = 112087.735.
Loss at epoch [12500/100000]: Ntot = 1042432.690, Nqll = 52743.568.
Loss at epoch [15000/100000]: Ntot = 833106.070, Nqll = 183962.174.
Loss at epoch [17500/100000]: Ntot = 768356.931, Nqll = 166522.075.
Loss at epoch [20000/100000]: Ntot = 712309.605, Nqll = 157763.136.
Loss at epoch [22500/100000]: Ntot = 651121.990, Nqll = 137999.885.
Loss at epoch [25000/100000]: Ntot = 694956.173, Nqll = 151152.165.
Loss at epoch [27500/100000]: Ntot = 688623.919, Nqll = 156523.017.
Loss at epoch [30000/100000]: Ntot = 754104.466, Nqll = 169974.494.
Loss at epoch [32500/100000]: Ntot = 768709.646, Nqll = 172686.379.
Loss at epoch [35000/100000]: Ntot = 775977.938, Nqll = 172839.736.
Los